<a href="https://colab.research.google.com/github/wagnerdgarcia/academicProjects/blob/master/Liga_de_Ferro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operacional`, de `Arenales, Armentano, Morabito e Yanasse`.</sup>

## Exercício 1























ção tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração, em termos percentuais, desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínima e máxima de cada componente da liga).

Frações dos elementos (%) nos ingredientes e custo dos ingredientes (R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.5 | 90 | 9 |
| Silício | 14 | - | 27 |
| Custo | 90 | 180 | 25 |

Frações (%) mínima e máxima dos componentes na liga:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 9.5 |
|Silício | 19 | 20 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [35]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Carrega Dados

In [36]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
}

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

In [37]:
a["l"]['c']

0.005

Cria modelo

In [38]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0) for i in range(3)]


model.objective = c["l"] * x[0] + c["g"] * x[1] + c["s"] * x[2]

model+= a["l"]["c"] * x[0] + a["g"]["c"] * x[1] + a["s"]["c"] * x[2] >= n["c"]*Q
model+= a["l"]["c"] * x[0] + a["g"]["c"] * x[1] + a["s"]["c"] * x[2] <= m["c"]*Q

model+=a["l"]["s"] * x[0] + a["g"]["s"] * x[1] + a["s"]["s"] * x[2] >= n["s"]*Q
model+=a["l"]["s"] * x[0] + a["g"]["s"] * x[1] + a["s"]["s"] * x[2] <= m["s"]*Q

model+=x[0] + x[1] + x[2] == Q

model.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0 + 180 x_1 + 25 x_2
Subject To
constr(0):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 0.95000
constr(2):  0.14000 x_0 + 0.27000 x_2 >= 1.90000
constr(3):  0.14000 x_0 + 0.27000 x_2 <= 2
constr(4):  x_0 + x_1 + x_2 = 10
Bounds
End



Executa

In [39]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")
  
  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
x_0 = 5.38
x_1 = 0.00
x_2 = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [40]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [41]:
model += x[0] <= e["l"]
model += x[1] <= e["g"]
model += x[2] <= e["s"]

model.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0 + 180 x_1 + 25 x_2
Subject To
constr(0):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 0.95000
constr(2):  0.14000 x_0 + 0.27000 x_2 >= 1.90000
constr(3):  0.14000 x_0 + 0.27000 x_2 <= 2
constr(4):  x_0 + x_1 + x_2 = 10
constr(5):  x_0 <= 5
constr(6):  x_1 <= 5
constr(7):  x_2 <= 12
Bounds
End



Executa

In [42]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
x_0 = 5.00
x_1 = 0.19
x_2 = 4.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e sua composição mínima e máxima é dada na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 0.4 |
|Silício | 0.12 | 0.19 |


### Código

Carrega dados

In [43]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

In [44]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=CONTINUOUS, name=f"x_{i}", lb=0.0) for i in range(6)]

# Funcao Objetivo
model.objective = c["l"] * x[0] + c["g"] * x[1] + c["s"] * x[2] + c["l"] * x[3] + c["g"] * x[4] + c["s"] * x[5]

# Restricao de carbono da primeira liga
model+= a["l"]["c"] * x[0] + a["g"]["c"] * x[1] + a["s"]["c"] * x[2] >= n[0]["c"]*Q[0]
model+= a["l"]["c"] * x[0] + a["g"]["c"] * x[1] + a["s"]["c"] * x[2] <= m[0]["c"]*Q[0]

# Restricao de carbono da segunda liga
model+= a["l"]["c"] * x[3] + a["g"]["c"] * x[4] + a["s"]["c"] * x[5] >= n[1]["c"]*Q[1]
model+= a["l"]["c"] * x[3] + a["g"]["c"] * x[4] + a["s"]["c"] * x[5] <= m[1]["c"]*Q[1]

# Restricao de silicio da Primeira liga
model+=a["l"]["s"] * x[0] + a["g"]["s"] * x[1] + a["s"]["s"] * x[2] >= n[0]["s"]*Q[0]
model+=a["l"]["s"] * x[0] + a["g"]["s"] * x[1] + a["s"]["s"] * x[2] <= m[0]["s"]*Q[0]

# Restricao de Silicio da segunda liga
model+=a["l"]["s"] * x[3] + a["g"]["s"] * x[4] + a["s"]["s"] * x[5] >= n[1]["s"]*Q[1]
model+=a["l"]["s"] * x[3] + a["g"]["s"] * x[4] + a["s"]["s"] * x[5] <= m[1]["s"]*Q[1]

# Quantidade em Toneladas de cada elemento
# primeira liga
model+=x[0] + x[1] + x[2] == Q[0]
# segunda liga
model+=x[3] + x[4] + x[5] == Q[1]

# Restricao do estoque
model += x[0] + x[3] <= e["l"]
model += x[1] + x[4] <= e["g"]
model += x[2] + x[5] <= e["s"]

model.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0 + 180 x_1 + 25 x_2 + 90 x_3 + 180 x_4 + 25 x_5
Subject To
constr(0):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 >= -0
constr(1):  0.00500 x_0 + 0.90000 x_1 + 0.09000 x_2 <= 0.95000
constr(2):  0.00500 x_3 + 0.90000 x_4 + 0.09000 x_5 >= -0
constr(3):  0.00500 x_3 + 0.90000 x_4 + 0.09000 x_5 <= 2.40000
constr(4):  0.14000 x_0 + 0.27000 x_2 >= 1.90000
constr(5):  0.14000 x_0 + 0.27000 x_2 <= 2
constr(6):  0.14000 x_3 + 0.27000 x_5 >= 0.72000
constr(7):  0.14000 x_3 + 0.27000 x_5 <= 1.14000
constr(8):  x_0 + x_1 + x_2 = 10
constr(9):  x_3 + x_4 + x_5 = 6
constr(10):  x_0 + x_3 <= 5
constr(11):  x_1 + x_4 <= 5
constr(12):  x_2 + x_5 <= 12
Bounds
End



Executa

In [45]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 1029.26

Solution:
x_0 = 4.32
x_1 = 0.51
x_2 = 5.17
x_3 = 0.68
x_4 = 1.45
x_5 = 3.87
